# Master Thesis 2022

## Master of Science in Engineering, Geomatik

### Fabian Escribano

In [ ]:
# ---- import libraries
import cv2
import numpy as np
from PIL import Image
from vidstab import VidStab
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops
from skimage.filters import threshold_otsu

# ---- import DeepLab
from model import DeepLabModel, DeepLab

In [ ]:
# ---- input video file path
video = str('./input/video/DJI_0008_short.mp4')

# ---- input foto file path
foto = 'input/foto/test.png'

# ---- output foto file path
output_foto = 'output/foto/cars.JPG'

# ---- output video file path
output_video = str('./output/video/DJI_0008_short.avi') # Path to Output-Video

# ---- output stabilized video path
video_stabilized = str('./output/video/test_output_short_stabilized.avi') # Path to stabilized Video

# ---- Frames per Second for Output Video File
fps = float(30.0)

In [ ]:
# ----Set basic variables
current_frame = 0

## StabilizeVideo

# Stabilize Video and save to File "video_stabilized"
stabilizer = VidStab(kp_method='FAST', threshold=42, nonmaxSuppression=False)
stabilizer.stabilize(input_path = video, output_path = video_stabilized, border_type = 'black', border_size=100)

stabilizer.plot_trajectory()
plt.show()

stabilizer.plot_transforms()
plt.show()


# Run DeepLab

In [ ]:
# ---- deeplab definitions

def create__label_colormap():
     return np.asarray([
         [0, 0, 0],
         [0, 192, 0],
         [255, 192, 192],
     ])
    

def label_to_color_image(label):
    """Adds color defined by the dataset colormap to the label.

    Args:
    label: A 2D array with integer type, storing the segmentation label.

    Returns:
    result: A 2D array with floating type. The element of the array
      is the color indexed by the corresponding element in the input label
      to the PASCAL color map.

    Raises:
    ValueError: If label is not of rank 2 or its value is larger than color
      map maximum entry.
    """
    if label.ndim != 2:
        raise ValueError('Expect 2-D input label')

    colormap = create__label_colormap()

    if np.max(label) >= len(colormap):
        raise ValueError('label value too large.')

    return colormap[label]

In [ ]:
# ---- iterate DeepLab trough images

model = DeepLabModel('model/frozen_inference_graph_cars.pb')

cap = cv2.VideoCapture(video)
cap.set(cv2.CAP_PROP_POS_AVI_RATIO,0)

img_array = []

while True:
    current_frame += 1
    ret, frame = cap.read()
    
    if frame is None:
        print('finished!')
        break
        
    mask = model.run(Image.fromarray(frame))
    
    label_img = label(mask)
    img_array.append(label_to_color_image(mask).astype(np.uint8))


In [ ]:
# ---- translate array
size = img_array[0].shape[:2][::-1]

In [ ]:
# ---- save to video
out = cv2.VideoWriter(output_video,cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()